# Experiments with Runnables

Create a few RunnableLambda, and compose them in sequence and in parralel

In [ ]:
from langchain_core.runnables import RunnablePassthrough, chain,  RunnableParallel, RunnableLambda
from devtools import debug
from asyncio import wait

add_1 = RunnableLambda(lambda x : x+1) 
add_3 = RunnableLambda(lambda x : x+3) 

@chain
def mult_2(x:int):
    return x*2

sequence = mult_2 | add_1
parallel = mult_2 | {"add_1": add_1, "add_3": add_3}

# same as above:
parallel1 = RunnableParallel(
    add_1=add_1,
    add_3=add_3
)


In [ ]:
parallel.invoke(1)

Run the runnable directly, batched, in parallel, ... 

In [ ]:
sequence.invoke(1) # 3
sequence.batch([1, 2, 3]) # [3,5,7]
parallel.invoke(1) # {'add_1': 3, 'add_2': 4}

await sequence.abatch([1,2,3,4,5])


Print the graph and various type information

In [ ]:
parallel.get_graph().print_ascii()

print("input type:",sequence.InputType)
print("output type:", sequence.OutputType)

print("input schema: ", sequence.input_schema().schema())
print("output schema: ",sequence.output_schema().schema())

Use RunnablePassthrough

In [ ]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=add_1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}



Demo 'bind' and 'RunnableConfig' 

In [ ]:
from langchain_core.runnables import RunnableConfig
from loguru import logger

@chain
def max(x:int, max:int, config: RunnableConfig) -> int:
    if log := (config["logger"]):
        log.info(f"check {x} < {max}")
    return max if x >= max else x

a = sequence | max.bind(max=3)
a.batch([1,2,3,4,5], {"logger": logger})

Demo 'assign"

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})